In [ ]:
# default_exp env

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Environment

In [ ]:
#export
import gym
from gym.spaces import Tuple, Discrete, Box
import numpy as np

class CarEnv(gym.Env):
    """
    Actions:
    0: accelerate
    1: break
    2: keep velocity
    3: declutch
    
    Observationspace
    1: distance travelled in meters
    2: distance to go in meters
    3: current velocity
    
    Rewards:
    - standing still     -1
    - goal reached      100
    - overshoot       -1000
    - closer to target    0

    """
    def __init__(self):
        super(CarEnv, self).__init__()
        
        # define distance
        # todo: define random distance
        self.distance: float = 1000.0
        self.maxspeed: float = 35.0 # maxspeed 35/ms
        self.velocityenergy_unit: float = 10.0 # 1 second at the speed of velocityenergy_unit uses 1 energy unit
        self.accelerationenergy_factor: float = 0.05 # acceleration uses an extra amount of engergy   
        self.max_timesteps = int(self.distance)
                
        self.currentposition: float = 0.0
        self.currentvelocity: float = 0.0
        self.usedenergy: float = 0.0
        self.is_done: bool = False
        
        # definition of observation value array
        low = np.array([0.0,
                        0.0,
                        0.0],
                       dtype=np.float32
        )
        
        high = np.array([self.distance,
                         self.distance,
                         self.maxspeed],
                         dtype=np.float32
        )
        
        self.observation_space = Box(low, high, dtype=np.float32)
        
        self.action_space = Discrete(n=4)
        
        self.step_index = 0

        
    def reset(self):
        self.currentposition = 0.0
        self.currentvelocity = 0.0
        self.usedenergy = 0.0
        self.is_done = False
        self.step_index = 0
        
        return self._calculate_state()
    
    def _calculate_state(self):
        return np.array([self.currentposition,
                         self.distance - self.currentposition,
                         self.currentvelocity
                        ], dtype=np.float32)
        
    def step(self, action):
        zero_state = np.array([0.0, 0.0, 0.0, 0.0], dtype=np.float32)
        if self.is_done:
            return zero_state, 0, self.is_done
        
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action), err_msg
        
        energy_for_step:float = 0.0
        reward = 0
        
        if action==0:
            # accelerate
            self.currentvelocity = min(self.maxspeed, self.currentvelocity + 1.0)
            energy_for_step = (self.currentvelocity / self.velocityenergy_unit) * (1 + self.accelerationenergy_factor)
            
        if action==1:
            # break
            self.currentvelocity = max(0.0, self.currentvelocity - 1.0)
            energy_for_step = 0.0 # using the breaks doesn't need energy
            
        if action==2:
            # keep velocity
            energy_for_step = (self.currentvelocity / self.velocityenergy_unit)
            
        if action==3:
            # declutch
            self.currentvelocity = max(0.0, self.currentvelocity - 0.1)
            energy_for_step = 0.0 # declutch doesn't use energy
        
        self.currentposition += self.currentvelocity
        self.step_index      += 1
        self.usedenergy      += energy_for_step
        
        goal_reached = bool(self.currentposition > self.distance -1 
                           and self.currentposition <= self.distance
                           and abs(self.currentvelocity - 0.0)<0.00001)
        
        overshoot = self.currentposition > self.distance
        
        if overshoot:
            reward = -1000
            self.is_done = True
            return zero_state, reward, self.is_done, {}
            
        if goal_reached:
            self.is_done = True
            reward = 100
            return zero_state, reward, self.is_done, {}
        
        if abs(self.currentvelocity - 0.0)<0.00001: 
            reward = -1
  
        return self._calculate_state(), reward, self.is_done, {}

In [ ]:
def test_init():
    env = CarEnv()
    result = env.reset()
    
    assert result[0] == 0.0
    assert result[1] == env.distance
    assert result[2] == 0.0

In [ ]:
def test_accelerate():
    env = CarEnv()
    env.reset()
    
    state, reward, done, _ = env.step(0)
    
    assert reward == 0
    assert done == False
    assert state[0] == 1.0
    assert state[1] == 999.0
    assert state[2] == 1.0

In [ ]:
def test_break():
    env = CarEnv()
    env.reset()
    
    env.step(0)
    state, reward, done, _ = env.step(1)
    
    assert reward == -1
    assert done == False
    assert state[0] == 1.0
    assert state[1] == 999.0
    assert state[2] == 0.0

In [ ]:
def test_keep_velocity():
    env = CarEnv()
    env.reset()
    env.step(0)
    
    state, reward, done, _ = env.step(2)
    
    assert reward == 0
    assert done == False
    assert state[0] == 2.0
    assert state[1] == 998.0
    assert state[2] == 1.0

In [ ]:
def test_declutch():
    env = CarEnv()
    env.reset()
    env.step(0)
    
    
    state, reward, done, _ = env.step(3)
    
    assert reward == 0
    assert done == False
    assert abs(state[0] - 1.9)<0.001
    assert abs(state[1] - 998.1)<0.001
    assert abs(state[2] - 0.9)<0.001

In [ ]:
test_init()
test_accelerate()
test_break()
test_keep_velocity()
test_declutch()